In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWhqtQ7EdUk-IGamlYiRl-yrSU2ei0FYCiXZWTlaZQoLkzVv8RV770Y



Successfully saved authorization token.


In [6]:
import ee
import geemap

Map = geemap.Map()

geometry = ee.Geometry.Polygon(
        [[[25.67758947139908, -10.52500539578282],
          [25.67758947139908, -10.831343855184292],
          [26.285562854675604, -10.831343855184292],
          [26.285562854675604, -10.52500539578282]]], None, False)
mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
gsw = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")

Map.centerObject(geometry, 11)
beforeStart = '2019-06-01'
beforeEnd = '2020-01-01'
afterStart = '2021-06-01'
afterEnd = '2022-01-01'

beforeComp = ee.Image(ee.ImageCollection("COPERNICUS/S2_SR") \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
                      .filter(ee.Filter.intersects('.geo', geometry)) \
                      .filterDate(ee.Date(beforeStart), ee.Date(beforeEnd)) \
                      .median() \
                      .clip(geometry))
afterComp = ee.Image(ee.ImageCollection("COPERNICUS/S2_SR") \
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
                      .filter(ee.Filter.intersects('.geo', geometry)) \
                      .filterDate(ee.Date(afterStart), ee.Date(afterEnd)) \
                      .median() \
                      .clip(geometry))

rgbVis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
}

Map.addLayer(beforeComp, rgbVis, '2019')
Map.addLayer(afterComp, rgbVis, '2021')

collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
  .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(geometry) \
  .select('VH')

beforeCollection = collection.filterDate(beforeStart, beforeEnd)
afterCollection = collection.filterDate(afterStart,afterEnd)

before = beforeCollection.mosaic().clip(geometry)
after = afterCollection.mosaic().clip(geometry)

Map.addLayer(before, {'min':-25, 'max':0}, 'Before')
Map.addLayer(after, {'min':-25, 'max':0}, 'After')

##### NOTE #####
# This version did not include a speckle filter

# ndvi
ndviBefore = beforeComp.normalizedDifference(['B8', 'B4']).rename(['ndvi']); # NIR, RED
ndviAfter = afterComp.normalizedDifference(['B8', 'B4']).rename(['ndvi']); # NIR, RED

cpThreshold = 8
permanentWater = gsw.select('seasonality').gte(7).clip(geometry)

# 2019
waterStorage2019 = before.lt(-22).rename('water').selfMask()
connectW2019 = waterStorage2019.connectedPixelCount(25)
waterStorage2019 = waterStorage2019.updateMask(connectW2019.gt(cpThreshold))
Map.addLayer(waterStorage2019, {'min':0, 'max':1, 'palette': ['A9D3FE']}, '2019 Water Storage')

vv2019 = before.gte(-22).And(before.lt(-18)).rename('mine').selfMask()
vv2019 = vv2019.where(permanentWater,0).selfMask()
connectVV2019 = vv2019.connectedPixelCount(25)
vv2019 = vv2019.updateMask(connectVV2019.gt(cpThreshold))
Map.addLayer(vv2019, {'min':0, 'max':1, 'palette': ['pink']}, '2019 Initial Mining Estimate')

mining2019 = before.gte(-22).And(before.lt(-18)).And(ndviBefore.lt(.1)).rename('mine').selfMask()
mining2019 = mining2019.where(permanentWater,0).selfMask()
connectM2019 = mining2019.connectedPixelCount(25)
mining2019 = mining2019.updateMask(connectM2019.gt(cpThreshold))
Map.addLayer(mining2019, {'min':0, 'max':1, 'palette': ['purple']}, '2019 Final Mining Estimate')

# 2021
waterStorage2021 = after.lt(-22).rename('water').selfMask()
connectW2021 = waterStorage2021.connectedPixelCount(25)
waterStorage2021 = waterStorage2021.updateMask(connectW2021.gt(cpThreshold))
Map.addLayer(waterStorage2021, {'min':0, 'max':1, 'palette': ['blue']}, '2021 Water Storage')

vv2021 = after.gte(-22).And(after.lt(-18)).rename('mine').selfMask()
vv2021 = vv2021.where(permanentWater,0).selfMask()
connectVV2021 = vv2021.connectedPixelCount(25)
vv2021 = vv2021.updateMask(connectVV2021.gt(cpThreshold))
Map.addLayer(vv2021, {'min':0, 'max':1, 'palette': ['orange']}, '2021 Initial Mining Estimate')

mining2021 = after.gte(-22).And(after.lt(-18)).And(ndviAfter.lt(.1)).rename('mine').selfMask()
mining2021 = mining2021.where(permanentWater,0).selfMask()
connectM2021 = mining2021.connectedPixelCount(25)
mining2021 = mining2021.updateMask(connectM2021.gt(cpThreshold))
Map.addLayer(mining2021, {'min':0, 'max':1, 'palette': ['red']}, '2021 Final Mining Estimate')
Map


Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…